**Nota**: Se recomienda ejecutar este notebook en Google Colab para asegurar la compatibilidad y evitar problemas de dependencias. El entrenamiento de los modelos requiere una cantidad significativa de memoria RAM y potencia de cómputo, que puede no estar disponible en todos los entornos locales.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/antoniotrapote/chord-prediction-tfm/blob/main/anexos/notebooks/03_modelado/03_modelo_gru.ipynb)
[![View on GitHub](https://img.shields.io/badge/View_on-GitHub-black?logo=github)](https://github.com/antoniotrapote/chord-prediction-tfm/blob/main/anexos/notebooks/03_modelado/03_modelo_gru.ipynb)

# Gated Recurrent Unit (GRU) model - Pytorch

Hemos utilizado PyTorch para implementar y entrenar un modelo de red neuronal recurrente basado en Gated Recurrent Units (GRU) para la predicción de acordes en secuencias musicales.

El último dataset utilizado fue `songdb_funcional_v4`

Contenido del notebook:
1. Entorno (Colab) - comprobación de versiones
2. Configuración de parámetros
3. Traer el CSV a Colab


In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter, defaultdict
from pathlib import Path
from typing import List, Tuple, Dict

## 1) Entorno (Colab)
El modelo fue entrenado en Google Colab, con las siguientes especificaciones:
>Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]  
>PyTorch: 2.8.0+cu126  
>CUDA disponible: True  
>GPU: Tesla T4

In [2]:
#@title Comprobar GPU/versions
import sys, torch
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ Activa GPU: Runtime ▶ Change runtime type ▶ GPU")


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
PyTorch: 2.8.0+cu126
CUDA disponible: True
GPU: Tesla T4


## 2) Traer el CSV desde GitHub
Descargamos directamente el dataset procesado desde el repositorio.

In [ ]:
import urllib.request

# Configuración para descargar el dataset desde GitHub
USER = "antoniotrapote"
REPO = "chord-prediction-tfm"
BRANCH = "main"
PATH_IN_REPO = "anexos/data/songdb_funcional_v4.csv"
URL = f"https://raw.githubusercontent.com/{USER}/{REPO}/{BRANCH}/{PATH_IN_REPO}"

# Ruta local donde guardar el archivo
data_path = "/content/songdb_funcional_v4.csv"

# Descargar el archivo CSV desde GitHub
urllib.request.urlretrieve(URL, data_path)
print(f"Dataset descargado en: {data_path}")

('/content/songdb_funcional_v4.csv',
 <http.client.HTTPMessage at 0x7bf810b7b920>)

## 3) Cargar CSV y tokenizar (whitespace)

In [ ]:
import pandas as pd, ast, re

# Parámetros de filtrado
sequence_col = "funcional_prog"  # Columna que contiene las secuencias de acordes
min_seq_len = 8  # Ignorar secuencias muy cortas

# Cargar el dataset
df = pd.read_csv(data_path)
assert sequence_col in df.columns, f"Columna {sequence_col} no encontrada en el CSV."
print("Filas totales:", len(df))
display(df[[sequence_col]].head(3))

def parse_tokens_simple(s: str):
    """Convierte la secuencia de acordes en una lista de tokens"""
    if isinstance(s, str) and s.strip().startswith("[") and s.strip().endswith("]"):
        try:
            lst = ast.literal_eval(s)
            if isinstance(lst, list):
                return [str(t) for t in lst]
        except Exception:
            pass
    # Normaliza separadores de compás y saltos de línea a espacios
    s = str(s).replace("|", " ").replace("\n", " ")
    toks = [t for t in re.findall(r"\S+", s) if t.strip()]
    return toks

# Tokenizar y filtrar secuencias muy cortas
df["_tokens_"] = df[sequence_col].apply(parse_tokens_simple)
df = df[df["_tokens_"].apply(len) >= min_seq_len].reset_index(drop=True)
print(f"Filas tras filtro min_seq_len >= {min_seq_len}:", len(df))

Filas totales: 2613


,funcional_prog
0,vi #ivø V/III V/VI vi IV ii V7 iii vi ii V7 I ...
1,VII VII I vi ii V7 VII VII I vi ii V7 I IV #iv...
2,i VI V/V V7 i VI V/V V7 i VI iiø V7 i VI iiø V...


Filas tras filtro min_seq_len: 2612


## 4) Split train/val/test (simple, por filas)

In [ ]:
from sklearn.model_selection import train_test_split

# Parámetros de división del dataset
val_size = 0.10     # 10% para validación
test_size = 0.10    # 10% para test
random_state = 42   # Semilla para reproducibilidad

# Dividir en train/val/test
train_df, tmp_df = train_test_split(df, test_size=val_size+test_size, random_state=random_state, shuffle=True)
rel_test = test_size / (val_size + test_size) if (val_size + test_size) > 0 else 0.5
val_df, test_df = train_test_split(tmp_df, test_size=rel_test, random_state=random_state, shuffle=True)

# Extraer las secuencias tokenizadas
train_seqs = train_df["_tokens_"].tolist()
val_seqs   = val_df["_tokens_"].tolist()
test_seqs  = test_df["_tokens_"].tolist()

print(f"Train: {len(train_seqs)}, Val: {len(val_seqs)}, Test: {len(test_seqs)}")

2089 261 262


## 5) Vocabulario y codificación

In [ ]:
from collections import Counter
import json

# Tokens especiales
PAD, UNK, BOS, EOS = "<pad>", "<unk>", "<bos>", "<eos>"
tokenizer_path = "gru_tokenizer.json"  # Nombre del archivo del tokenizer

def build_vocab(seqs, min_freq=1):
    """Construye el vocabulario a partir de las secuencias de entrenamiento"""
    c = Counter()
    for s in seqs: 
        c.update(s)
    
    # Crear vocabulario: tokens especiales + tokens frecuentes
    vocab = [PAD, UNK, BOS, EOS] + [t for t,f in c.items() if f >= min_freq and t not in {PAD,UNK,BOS,EOS}]
    stoi = {t:i for i,t in enumerate(vocab)}  # string to index
    itos = {i:t for t,i in stoi.items()}     # index to string
    return vocab, stoi, itos

# Construir vocabulario solo con datos de entrenamiento
vocab, stoi, itos = build_vocab(train_seqs, min_freq=1)
vocab_size = len(vocab)
print("Tamaño del vocabulario:", vocab_size)

# Guardar el tokenizer para uso posterior
with open(tokenizer_path, "w") as f:
    json.dump({"vocab": vocab}, f, ensure_ascii=False, indent=2)
print(f"Tokenizer guardado en: {tokenizer_path}")

def encode(seq, add_bos=True):
    """Convierte una secuencia de tokens a índices numéricos"""
    ids = [stoi[BOS]] if add_bos else []
    ids += [stoi.get(t, stoi[UNK]) for t in seq]
    return ids

Vocab size: 86


## 6) Dataset (context→next) y DataLoaders

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Parámetros del modelo y entrenamiento
seq_len = 24        # Longitud de secuencia de contexto
batch_size = 128    # Tamaño del batch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class NextTokenDataset(Dataset):
    """Dataset que genera pares (contexto, siguiente_token) para entrenamiento"""
    def __init__(self, sequences, seq_len):
        self.samples = []
        for seq in sequences:
            ids = encode(seq, add_bos=True)
            if len(ids) <= seq_len: 
                continue
            # Crear ventanas deslizantes de seq_len -> next_token
            for i in range(seq_len, len(ids)):
                self.samples.append((ids[i-seq_len:i], ids[i]))
    
    def __len__(self): 
        return len(self.samples)
    
    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

# Crear datasets
train_data = NextTokenDataset(train_seqs, seq_len)
val_data   = NextTokenDataset(val_seqs, seq_len)
test_data  = NextTokenDataset(test_seqs, seq_len)

# Crear DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print(f"Muestras de entrenamiento: {len(train_data)}")
print(f"Muestras de validación: {len(val_data)}")
print(f"Muestras de test: {len(test_data)}")

(60979, 7044, 7669)

## 7) Modelo GRU

In [ ]:
import torch.nn as nn

class ChordGRU(nn.Module):
    """
    Modelo GRU para predicción de acordes musicales.
    Arquitectura: Embedding -> GRU -> Dropout -> Linear
    """
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_size, num_layers=num_layers,
                          batch_first=True, dropout=dropout if num_layers>1 else 0.0)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x):
        # x: (batch_size, seq_len)
        e = self.emb(x)                    # (batch_size, seq_len, embedding_dim)
        o, _ = self.rnn(e)                 # (batch_size, seq_len, hidden_size)
        return self.fc(self.dropout(o[:, -1, :]))  # Solo último timestep -> (batch_size, vocab_size)

# Parámetros del modelo (manteniendo los valores originales)
embedding_dim = 128     # Dimensión de los embeddings
hidden_size = 256       # Tamaño oculto del GRU
num_layers = 2          # Número de capas GRU
dropout = 0.2           # Tasa de dropout

## 8) Entrenamiento y métricas (Top@K, MRR, PPL)

In [ ]:
import math, time, os, torch
import torch.nn.functional as F

def topk_metrics(logits, targets, ks=(1,3,5)):
    """Calcula métricas Top@K y MRR"""
    out = {}
    with torch.no_grad():
        for k in ks:
            topk = logits.topk(k, dim=-1).indices
            out[f"Top@{k}"] = (topk == targets.unsqueeze(1)).any(dim=1).float().mean().item()
        # Mean Reciprocal Rank
        ranks = (logits.argsort(dim=-1, descending=True) == targets.unsqueeze(1)).nonzero(as_tuple=False)[:,1] + 1
        out["MRR"] = (1.0 / ranks.float()).mean().item()
    return out

def evaluate(model, loader, criterion):
    """Evalúa el modelo en un conjunto de datos"""
    model.eval()
    total, n = 0.0, 0
    agg = {"Top@1":0.0,"Top@3":0.0,"Top@5":0.0,"MRR":0.0}
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            b = x.size(0); total += loss.item()*b; n += b
            m = topk_metrics(logits, y)
            for k in agg: agg[k] += m[k]*b
    for k in agg: agg[k] /= max(1,n)
    return {"loss": total/max(1,n), "ppl": math.exp(total/max(1,n)), **agg}

def train_model(model, train_loader, val_loader, epochs, lr, weight_decay, grad_clip=1.0, amp=True, save_dir=".", save_name="best.pt"):
    """Entrena el modelo GRU"""
    os.makedirs(save_dir, exist_ok=True)
    scaler = torch.cuda.amp.GradScaler(enabled=(amp and device.type=='cuda'))
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    crit = torch.nn.CrossEntropyLoss()
    best_mrr, best_path = -1.0, os.path.join(save_dir, save_name)
    
    for ep in range(1, epochs+1):
        model.train(); t0 = time.time()
        for i,(x,y) in enumerate(train_loader,1):
            x,y = x.to(device), y.to(device)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=(amp and device.type=='cuda')):
                logits = model(x); loss = crit(logits,y)
            scaler.scale(loss).backward()
            if grad_clip is not None:
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(opt); scaler.update()
            if i % 100 == 0: 
                print(f"Ep{ep} step {i}/{len(train_loader)} loss {loss.item():.4f}")
        
        # Evaluación en validación
        valm = evaluate(model, val_loader, crit)
        print(f"Epoch {ep} | val loss {valm['loss']:.4f} ppl {valm['ppl']:.2f} Top@1 {valm['Top@1']:.3f} Top@3 {valm['Top@3']:.3f} Top@5 {valm['Top@5']:.3f} MRR {valm['MRR']:.3f}")
        
        # Guardar mejor modelo basado en MRR
        if valm["MRR"] > best_mrr:
            best_mrr = valm["MRR"]
            torch.save({"model_state": model.state_dict(), "stoi": stoi, "itos": itos}, best_path)
            print("🔥 Guardado best ->", best_path, "| MRR:", best_mrr)
    return best_mrr, best_path

## 9) Entrenar GRU

In [ ]:
import torch, os

# Parámetros de entrenamiento (valores originales del experimento)
epochs = 6              # Número de épocas
lr = 2e-3              # Learning rate (0.002)
weight_decay = 1e-4    # Regularización L2 (0.0001)
grad_clip = 1.0        # Gradient clipping
amp = True             # Automatic Mixed Precision
random_state = 42      # Semilla para reproducibilidad

# Rutas de guardado
save_dir = "/content/models_gru_v2"
save_name = "gru_best.pt"

# Fijar semilla para reproducibilidad
torch.manual_seed(random_state)

# Crear modelo
model = ChordGRU(vocab_size=len(vocab), 
                 embedding_dim=embedding_dim,
                 hidden_size=hidden_size, 
                 num_layers=num_layers,
                 dropout=dropout).to(device)

print(f"Modelo creado con {sum(p.numel() for p in model.parameters())} parámetros")

# Entrenar modelo
best_mrr, best_path = train_model(model, train_loader, val_loader, 
                                  epochs=epochs, lr=lr, weight_decay=weight_decay,
                                  grad_clip=grad_clip, amp=amp, 
                                  save_dir=save_dir, save_name=save_name)

print(f"🎯 Mejor MRR: {best_mrr:.4f} | Modelo guardado en: {best_path}")

/tmp/ipython-input-1507830936.py:32: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(amp and device.type=='cuda'))
/tmp/ipython-input-1507830936.py:41: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(amp and device.type=='cuda')):


Ep1 step 100/476 loss 2.2282
Ep1 step 200/476 loss 2.3747
Ep1 step 300/476 loss 2.0149
Ep1 step 400/476 loss 2.0779
Epoch 1 | val loss 2.1584 ppl 8.66 Top@1 0.442 Top@3 0.676 Top@5 0.763 MRR 0.585
🔥 Guardado best -> /content/models_gru_v1/gru_best.pt | MRR: 0.584794789535765
Ep2 step 100/476 loss 2.1084
Ep2 step 200/476 loss 2.1919
Ep2 step 300/476 loss 2.1190
Ep2 step 400/476 loss 2.1181
Epoch 2 | val loss 2.1306 ppl 8.42 Top@1 0.448 Top@3 0.685 Top@5 0.768 MRR 0.591
🔥 Guardado best -> /content/models_gru_v1/gru_best.pt | MRR: 0.591246411382035
Ep3 step 100/476 loss 2.0119
Ep3 step 200/476 loss 1.8909
Ep3 step 300/476 loss 1.8901
Ep3 step 400/476 loss 2.1402
Epoch 3 | val loss 2.1259 ppl 8.38 Top@1 0.455 Top@3 0.688 Top@5 0.770 MRR 0.597
🔥 Guardado best -> /content/models_gru_v1/gru_best.pt | MRR: 0.5970695267882555
Ep4 step 100/476 loss 1.9535
Ep4 step 200/476 loss 1.8190
Ep4 step 300/476 loss 1.8945
Ep4 step 400/476 loss 2.0423
Epoch 4 | val loss 2.1204 ppl 8.33 Top@1 0.459 Top@3 0.

## 10) Evaluación en Test

In [ ]:
import torch, os

# Cargar el mejor modelo guardado
model_path = os.path.join(save_dir, save_name)
ckpt = torch.load(model_path, map_location=device)
model.load_state_dict(ckpt["model_state"])

print(f"✅ Modelo cargado desde: {model_path}")

# Evaluar en conjunto de test
test_metrics = evaluate(model, test_loader, torch.nn.CrossEntropyLoss())

print("📊 Métricas finales en Test:")
print(f"  • Loss: {test_metrics['loss']:.4f}")
print(f"  • Perplexity: {test_metrics['ppl']:.2f}")
print(f"  • Top@1: {test_metrics['Top@1']:.3f}")
print(f"  • Top@3: {test_metrics['Top@3']:.3f}")
print(f"  • Top@5: {test_metrics['Top@5']:.3f}")
print(f"  • MRR: {test_metrics['MRR']:.3f}")

Test: {'loss': 2.1969814152866194, 'ppl': 8.997811807607055, 'Top@1': 0.4319989573755014, 'Top@3': 0.6775329250729151, 'Top@5': 0.7616377626377533, 'MRR': 0.5797308985110382}


## 11) predict_next(context, k=5)

In [ ]:
import torch.nn.functional as F

def predict_next(context_tokens, k=5):
    """
    Predice los k acordes más probables dado un contexto.
    
    Args:
        context_tokens: Lista de tokens de acordes como contexto
        k: Número de predicciones a devolver
    
    Returns:
        Lista de tuplas (token, probabilidad)
    """
    model.eval()
    
    # Convertir tokens a índices
    ids = [stoi.get(t, stoi["<unk>"]) for t in context_tokens]
    
    # Ajustar longitud al seq_len esperado
    if len(ids) < seq_len:
        ids = [stoi["<bos>"]] * (seq_len - len(ids)) + ids
    else:
        ids = ids[-seq_len:]  # Tomar últimos seq_len tokens
    
    # Crear tensor y predecir
    x = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(0)
    with torch.no_grad():
        logits = model(x)
        probs = F.softmax(logits[0], dim=-1)
        topk = torch.topk(probs, k)
        return [(itos[i], float(p)) for i,p in zip(topk.indices.tolist(), topk.values.tolist())]

# Ejemplo de predicción con una secuencia del conjunto de entrenamiento
if len(train_seqs) > 0:
    # Tomar una secuencia de ejemplo
    ejemplo_seq = train_seqs[0][:seq_len-2]  # Dejar espacio para predicción
    print("🎵 Contexto de ejemplo:", " ".join(ejemplo_seq))
    
    # Predecir próximos acordes
    predicciones = predict_next(ejemplo_seq, k=5)
    print("🔮 Top 5 predicciones:")
    for i, (token, prob) in enumerate(predicciones, 1):
        print(f"  {i}. {token} ({prob:.3f})")

Context: ['III', 'III', '#IV', '#IV', 'bII', 'bII', 'natIII', 'natIII', 'III', '#IV', 'IV', 'VI', 'V', 'bVII', 'natVI', 'I', 'II', 'VII', 'VI', 'IV', 'III', 'I', 'VII', 'natVI']
Pred: [('VII', 0.13458140194416046), ('VI', 0.12113039195537567), ('I', 0.11372911930084229), ('IV', 0.08486910164356232), ('III', 0.07772155851125717)]



### Roadmap corto
- Añadir **posición en compás** y **duración** como embeddings adicionales (v2).
- Re-ranking suave para evitar repes y favorecer cadencias.
- Ajustar `seq_len`, capas y *scheduler* cuando confirmes el pipeline.
